In [1]:
import os
import pandas as pd
import numpy
from datetime import datetime, time

import warnings
warnings.filterwarnings("ignore")

In [2]:
file_path = "./Analytics Position Case Study.xlsx"

# Read all sheets into a dictionary of dataframes
excel_data = pd.read_excel(file_path, sheet_name=None, header = 3)

# Access each sheet by its name
# for sheet_name, data_frame in excel_data.items():
#     print(f"Sheet name: {sheet_name}")


In [3]:
User_Gameplay_data = excel_data["User Gameplay data"].copy()
User_Gameplay_data.columns = [x.lower().replace(" ", "_") for x in User_Gameplay_data.columns]
print(User_Gameplay_data.shape)

Deposit_Data = excel_data["Deposit Data"].copy()
Deposit_Data.columns = [x.lower().replace(" ", "_") for x in Deposit_Data.columns]
print(Deposit_Data.shape)

Withdrawal_Data = excel_data["Withdrawal Data"].copy()
Withdrawal_Data.columns = [x.lower().replace(" ", "_") for x in Withdrawal_Data.columns]
print(Withdrawal_Data.shape)


(355266, 3)
(17438, 3)
(3566, 3)


In [4]:
User_Gameplay_data["datetime"] = pd.to_datetime(User_Gameplay_data["datetime"])
Deposit_Data["datetime"] = pd.to_datetime(Deposit_Data["datetime"])
Withdrawal_Data["datetime"] = pd.to_datetime(Withdrawal_Data["datetime"])


In [5]:
# Based on the above information and the data provided answer the following questions:
# Questiion 1. Find Playerwise Loyalty points earned by Players in the following slots:-
#     a. 2nd October Slot S1
#     b. 16th October Slot S2
#     b. 18th October Slot S1
#     b. 26th October Slot S2

# Function to calculate loyalty points for a given date and time slot
def calculate_loyalty_points(date, slot):
    start_time, end_time = time_slots[slot]
    
    # Filter data for the given date and time slot
    deposit_data = Deposit_Data[
        (Deposit_Data['datetime'].dt.date == datetime.strptime(date, '%Y-%m-%d').date()) &
        (Deposit_Data['datetime'].dt.time >= start_time) &
        (Deposit_Data['datetime'].dt.time <= end_time)
    ]
    withdrawal_data = Withdrawal_Data[
        (Withdrawal_Data['datetime'].dt.date == datetime.strptime(date, '%Y-%m-%d').date()) &
        (Withdrawal_Data['datetime'].dt.time >= start_time) &
        (Withdrawal_Data['datetime'].dt.time <= end_time)
    ]
    gameplay_data = User_Gameplay_data[
        (User_Gameplay_data['datetime'].dt.date == datetime.strptime(date, '%Y-%m-%d').date()) &
        (User_Gameplay_data['datetime'].dt.time >= start_time) &
        (User_Gameplay_data['datetime'].dt.time <= end_time)
    ]
    
    # Calculate loyalty points for each player in the given date and time slot
    loyalty_points = {}
    for user_id in set(deposit_data['user_id']).union(set(withdrawal_data['user_id'])).union(set(gameplay_data['user_id'])):
        player_deposit = deposit_data[deposit_data['user_id'] == user_id]['amount'].sum()
        player_withdrawal = withdrawal_data[withdrawal_data['user_id'] == user_id]['amount'].sum()
        player_games_played = gameplay_data[gameplay_data['user_id'] == user_id]['games_played'].sum()

        loyalty_points[user_id] = (
            0.01 * player_deposit +
            0.005 * player_withdrawal +
            0.001 * max(player_deposit - player_withdrawal, 0) +
            0.2 * player_games_played
        )

    return loyalty_points

# Define the dates and time slots
dates = ['2022-10-02', '2022-10-16', '2022-10-18', '2022-10-26']
time_slots = {'S1': (time(0, 0, 0), time(11, 59, 59)),
            'S2': (time(12, 0, 0), time(23, 59, 59))}

# Calculate loyalty points for each player in the specified slots
loyalty_points = {}
for date in dates:
    for slot in time_slots.keys():
        loyalty_points[date + ' Slot ' + slot] = calculate_loyalty_points(date, slot)

loyalty_points = {key: value for key, value in loyalty_points.items() if value}
loyalty_points = pd.DataFrame(loyalty_points).fillna(0)

loyalty_points.reset_index(inplace = True)
loyalty_points.rename(columns = {"index": "user_id"}, inplace=True)

loyalty_points.head()


,user_id,2022-10-16 Slot S1,2022-10-16 Slot S2,2022-10-18 Slot S1,2022-10-18 Slot S2,2022-10-26 Slot S1,2022-10-26 Slot S2
0,2.0,0.2,0.4,0.4,440.2,0.2,990.000
1,3.0,0.2,0.0,0.4,0.2,0.6,0.000
2,5.0,6.9,12.8,1.6,8.9,22.6,11.800
3,7.0,0.6,0.0,0.6,0.2,0.0,33.285
4,8.0,2.2,1.0,1.8,1.0,1.8,1.400


In [8]:
# 2. Calculate overall loyalty points earned and rank players on the basis of loyalty points in the month of October. 
#      In case of tie, number of games played should be taken as the next criteria for ranking.

# Filter data for the month of October
october_start_date = pd.Timestamp('2022-10-01 00:00:00')
october_end_date = pd.Timestamp('2022-10-31 23:59:59')

october_deposit_data = Deposit_Data[
    (Deposit_Data['datetime'] >= october_start_date) &
    (Deposit_Data['datetime'] <= october_end_date)
]
october_withdrawal_data = Withdrawal_Data[
    (Withdrawal_Data['datetime'] >= october_start_date) &
    (Withdrawal_Data['datetime'] <= october_end_date)
]
october_gameplay_data = User_Gameplay_data[
    (User_Gameplay_data['datetime'] >= october_start_date) &
    (User_Gameplay_data['datetime'] <= october_end_date)
]


# ======= calculate loyalty points
user_ids_deposite = october_deposit_data.groupby("user_id").sum()
user_ids_withdraw = october_withdrawal_data.groupby("user_id").sum()

dep_minus_withdraw = user_ids_deposite - user_ids_withdraw
dep_minus_withdraw = dep_minus_withdraw["amount"]

games_played = october_gameplay_data.groupby("user_id").sum()

# Calculate loyalty points for October
loyalty_points = (
        0.01 * user_ids_deposite["amount"] + 
        0.005 * user_ids_withdraw["amount"] + 
        0.001 * pd.DataFrame(dep_minus_withdraw)["amount"] + 
        0.2 * games_played['games_played']
        )

october_ranked_players = pd.DataFrame(loyalty_points[loyalty_points.notna()], columns=["loyalty_points"])

# leaderboard = 
october_ranked_players["ranks"] = october_ranked_players["loyalty_points"].rank(ascending=False, method='min')

october_ranked_players.sort_values(by = "ranks", inplace = True)
october_ranked_players["games_played"] = games_played

october_ranked_players.reset_index(inplace = True)

october_ranked_players.head()


,user_id,loyalty_points,ranks,games_played
0,634.0,49707.808,1.0,22.0
1,672.0,14487.300,2.0,8.0
2,566.0,13328.320,3.0,106.0
3,99.0,12428.300,4.0,4.0
4,740.0,11981.352,5.0,2.0


In [6]:
# 3. What is the average deposit amount?
average_deposit_amount = Deposit_Data['amount'].mean()
print("Average deposit amount:", round(average_deposit_amount, 3))

Average deposit amount: 5492.185


In [7]:
# Question: 4. What is the average deposit amount per user in a month?

# Convert 'datetime' column to datetime type
Deposit_Data['datetime'] = pd.to_datetime(Deposit_Data['datetime'])

# Extract month from the 'datetime' column
Deposit_Data['month'] = Deposit_Data['datetime'].dt.month

# Calculate average deposit amount per user in a month
average_deposit_per_user = Deposit_Data.groupby('user_id')['amount'].mean()

# Display the average deposit amount per user
average_deposit_per_user = pd.DataFrame(average_deposit_per_user.round(3))
average_deposit_per_user.head(10)


,amount
user_id,
1.0,5000.000
2.0,28350.000
3.0,10000.000
4.0,1750.000
5.0,1105.970
6.0,1328.571
9.0,1776.917
10.0,1991.048
11.0,2204.762


In [11]:
# 5. What is the average number of games played per user?
average_games_played = User_Gameplay_data['games_played'].mean()
print("Average number of games played per user:", round(average_games_played, 3))


Average number of games played per user: 1.0


In [12]:
# "Part B - How much bonus should be allocated to leaderboard players?
# After calculating the loyalty points for the whole month find out which 50 players are at the top of the leaderboard. The company has allocated a pool of Rs 50000 to be given away as bonus money to the loyal players.
# Now the company needs to determine how much bonus money should be given to the players.
# Should they base it on the amount of loyalty points? Should it be based on number of games? Or something else?
# That’s for you to figure out.
# Suggest a suitable way to divide the allocated money keeping in mind the following points:
# 1. Only top 50 ranked players are awarded bonus

# Filter data for the month of October
october_start_date = pd.Timestamp('2022-10-01 00:00:00')
october_end_date = pd.Timestamp('2022-10-31 23:59:59')

october_deposit_data = Deposit_Data[
    (Deposit_Data['datetime'] >= october_start_date) &
    (Deposit_Data['datetime'] <= october_end_date)
]
october_withdrawal_data = Withdrawal_Data[
    (Withdrawal_Data['datetime'] >= october_start_date) &
    (Withdrawal_Data['datetime'] <= october_end_date)
]
october_gameplay_data = User_Gameplay_data[
    (User_Gameplay_data['datetime'] >= october_start_date) &
    (User_Gameplay_data['datetime'] <= october_end_date)
]


# ======= calculate loyalty points
user_ids_deposite = october_deposit_data.groupby("user_id").sum()
user_ids_withdraw = october_withdrawal_data.groupby("user_id").sum()

dep_minus_withdraw = user_ids_deposite - user_ids_withdraw
dep_minus_withdraw = dep_minus_withdraw["amount"]

games_played = october_gameplay_data.groupby("user_id").sum()

# Calculate loyalty points for October
loyalty_points = (
        0.01 * user_ids_deposite["amount"] + 
        0.005 * user_ids_withdraw["amount"] + 
        0.001 * pd.DataFrame(dep_minus_withdraw)["amount"] + 
        0.2 * games_played['games_played']
        )

loyalty_points = pd.DataFrame(loyalty_points[loyalty_points.notna()], columns=["loyalty_points"])

# leaderboard = 
loyalty_points["ranks"] = loyalty_points["loyalty_points"].rank(ascending=False, method='min')

loyalty_points.sort_values(by = "ranks", inplace = True)

top_50_ranks = loyalty_points[loyalty_points['ranks'].isin(loyalty_points['ranks'].nsmallest(50))]

# Calculate the total sum of loyalty points for the top 50 players
total_loyalty_points_top_50 = top_50_ranks["loyalty_points"].sum()

# Calculate the percentage contribution of each player's loyalty points to the total sum
top_50_ranks["percentage_contribution"] = top_50_ranks["loyalty_points"] / total_loyalty_points_top_50

top_50_ranks["bonus_allocation"] = top_50_ranks["percentage_contribution"] * 50000

top_50_ranks.reset_index(inplace = True)

top_50_ranks


,user_id,loyalty_points,ranks,percentage_contribution,bonus_allocation
0,634.0,49707.808,1.0,0.135696,6784.823308
1,672.0,14487.300,2.0,0.039549,1977.431206
2,566.0,13328.320,3.0,0.036385,1819.237255
3,99.0,12428.300,4.0,0.033928,1696.389821
4,740.0,11981.352,5.0,0.032708,1635.384049
5,30.0,11654.380,6.0,0.031815,1590.754295
6,369.0,10260.568,7.0,0.028010,1400.507158
7,992.0,9721.428,8.0,0.026538,1326.917720
8,365.0,9243.540,9.0,0.025234,1261.688820
9,421.0,9099.936,10.0,0.024842,1242.087719
